In [1]:
from numpy.random.bit_generator import SeedlessSeedSequence
import tensorflow as tf
import os
import numpy as np
import random
from sklearn.utils import shuffle
#from tqdm import tqdm #Provides a progress bar while the images are being resized
from skimage.io import imshow, imread
from skimage.transform import resize
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import tifffile as tiff 
from patchify import patchify, unpatchify
from sklearn.model_selection import train_test_split
from patchify import patchify
from sklearn.metrics import jaccard_score,  accuracy_score, f1_score, make_scorer, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import cross_val_score
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn import preprocessing
import random
#%load_ext tensorboard
from keras import backend as K
seed =42
np.random.seed=42
from spectral import *

In [2]:
HEIGHT = 512
WIDTH = 512
CHANNELS = 5

In [3]:
def average_calc(X_examine, y_examine):
  IoUs = [len(X_examine)]
  total =0
  average =0

  for i in range(0,len(X_examine)):
    test_predict = model.predict(np.expand_dims(X_examine[i],0))
    test_predict = np.asarray(test_predict) 
    IoU = jaccard_score((test_predict>0.5).squeeze(), (y_examine[i]>0.5).squeeze(), average='micro')
    total =IoU + total
  average = total / len(X_examine)
  
  return average

def average_Fscore(X_examine, y_examine):
  IoUs = [len(X_examine)]
  total =0
  average =0

  for i in range(0,len(X_examine)):
    test_predict = model.predict(np.expand_dims(X_examine[i],0))
    test_predict = np.asarray(test_predict) 
    IoU = f1_score((test_predict>0.6).squeeze(), (y_examine[i]).squeeze(), average='micro')
    total =IoU + total
  average = total / len(X_examine)
  
  return average

In [4]:
X=[]
y=[]

for a in range(1,6):
  Xpath = '/Users/anshugusain/tf-nuclei/content/Stained-40X/Field' + str(a) + '/cube_envi32.hdr'
  Ximg = spectral.open_image(Xpath)


  Field = Ximg.read_band((52,53,91,92,131))
  #Field = Ximg.read_bands((38,40,88,93,95,97,100,105,106,110)) # 10 band
  #Field = Ximg.read_bands((100,105))
  
  ypath = '/Users/anshugusain/tf-nuclei/content/Stained-40X/Field' + str(a) + '/Field' + str(a) + '-mask.tif'
  yimg = tf.keras.preprocessing.image.load_img(ypath, grayscale=True)
    

  Xpatches = patchify(Field,(HEIGHT,WIDTH,CHANNELS),step=HEIGHT)

  yimg = np.asarray(yimg)>0
  ypatches = patchify(yimg, (HEIGHT,WIDTH), step=HEIGHT)

  for m in range(0, Xpatches.shape[0]):
    for l in range(0, Xpatches.shape[1]):
      X.append(Xpatches[m,l,0])
      y.append(ypatches[m,l])

X = np.array(X)
y = np.array(y)

X = (X/X.max())
y = (y/y.max())

/Users/anshugusain/miniforge3/envs/tf-nuclei/lib/python3.9/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)
/Users/anshugusain/miniforge3/envs/tf-nuclei/lib/python3.9/site-packages/keras/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [5]:
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)
def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)  # -1 ultiplied as we want to minimize this value as loss function

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
import sklearn

In [8]:


#define cross-validation method to use
cv = LeaveOneOut()

In [9]:
for i,j in cv.split(X):
    print(i,j)

[1 2 3 4] [0]
[0 2 3 4] [1]
[0 1 3 4] [2]
[0 1 2 4] [3]
[0 1 2 3] [4]


In [10]:
X_test[0].shape

NameError: name 'X_test' is not defined

In [11]:
X_test.shape

NameError: name 'X_test' is not defined

In [12]:
y_true, y_pred = [], []

for i,j in cv.split(X):
    print(i,j)
    X_train, X_test = X[i,:],X[j,:]
    print(X_train)
    print(X_test)
    y_train, y_test = y[i], y[j]

    #Building the model

    inputs = tf.keras.layers.Input((HEIGHT, WIDTH, CHANNELS))
    s = inputs #converts integers from input layers into floating points

    #Contraction Path
                            # feature space  
                            # \/
    c1 = tf.keras.layers.Conv2D(16,(3,3), activation = "relu", kernel_initializer='he_normal', padding = 'same')(s)#kernel initializer is the initial weights used by the neural network (he_normal is a trunkated normal distribution centered on 0)
    c1 = tf.keras.layers.Dropout(.1)(c1)
    c1 = tf.keras.layers.Conv2D(16,(3,3), activation = "relu", kernel_initializer="he_normal", padding = "same")(c1 ) #Padding ensures that the output image has the same dimensions as the input image
    p1 = tf.keras.layers.MaxPooling2D((2,2))(c1) #First maxpooling layer puts the values into a 2x2 matrix, and gets the maximum values

    c2 = tf.keras.layers.Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p1)
    c2 = tf.keras.layers.Dropout(.1)(c2)
    c2 = tf.keras.layers.Conv2D(32,(3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c2)
    p2=tf.keras.layers.MaxPooling2D((2,2))(c2)

    c3= tf.keras.layers.Conv2D(64, (3,3), activation = "relu", kernel_initializer="he_normal", padding="same")(p2)
    c3 = tf.keras.layers.Dropout(.2)(c3)
    c3=tf.keras.layers.Conv2D(64,(3,3), activation = "relu", kernel_initializer="he_normal", padding="same")(c3)
    p3=tf.keras.layers.MaxPooling2D((2,2))(c3)

    c4= tf.keras.layers.Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p3)
    c4= tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3,3),activation="relu", kernel_initializer="he_normal", padding="same")(c4)
    p4=tf.keras.layers.MaxPooling2D((2,2))(c4)

    c5=tf.keras.layers.Conv2D(256, (3,3), activation ="relu", kernel_initializer="he_normal", padding="same")(p4)
    c5=tf.keras.layers.Dropout(0.3)(c5)
    c5=tf.keras.layers.Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c5)
    p5=tf.keras.layers.MaxPooling2D((2,2))(c5)

    c6=tf.keras.layers.Conv2D(512, (3,3), activation ="relu", kernel_initializer="he_normal", padding="same")(p5)
    c6=tf.keras.layers.Dropout(0.3)(c6)
    c6=tf.keras.layers.Conv2D(512, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c6)



    u8= tf.keras.layers.Conv2DTranspose(256, (2,2), strides=(2,2), padding="same")(c6)
    u8= tf.keras.layers.concatenate([u8,c5])#Concatenates layers u6 and c4 as shown in the Unet architecture
    c8=tf.keras.layers.Conv2D(256, (3,3), activation="relu", kernel_initializer='he_normal', padding="same")(u8)
    c8=tf.keras.layers.Dropout(0.2)(c8)
    c8=tf.keras.layers.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)


    u9= tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding="same")(c8)
    u9= tf.keras.layers.concatenate([u9,c4])#Concatenates layers u6 and c4 as shown in the Unet architecture
    c9=tf.keras.layers.Conv2D(128, (3,3), activation="relu", kernel_initializer='he_normal', padding="same")(u9)
    c9=tf.keras.layers.Dropout(0.2)(c9)
    c9=tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    u10= tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding="same")(c9)
    u10= tf.keras.layers.concatenate([u10,c3])#Concatenates layers u6 and c4 as shown in the Unet architecture
    c10=tf.keras.layers.Conv2D(64, (3,3), activation="relu", kernel_initializer='he_normal', padding="same")(u10)
    c10=tf.keras.layers.Dropout(0.2)(c10)
    c10=tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c10)

    u11=tf.keras.layers.Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(c10)
    u11=tf.keras.layers.concatenate([u11,c2])
    c11=tf.keras.layers.Conv2D(32,(3,3), activation="relu",kernel_initializer="he_normal", padding="same")(u11)
    c11=tf.keras.layers.Dropout(0.2)(c11)
    c11=tf.keras.layers.Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c11)

    u12=tf.keras.layers.Conv2DTranspose(16, (2,2),strides=(2,2), padding="same")(c11)
    u12=tf.keras.layers.concatenate([u12, c1])
    c12=tf.keras.layers.Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u12)
    c12=tf.keras.layers.Dropout(0.1)(c12)
    c12=tf.keras.layers.Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c12)


    outputs=tf.keras.layers.Conv2D(1, (1,1), activation="sigmoid")(c12)
    model=tf.keras.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(lr=.001), loss=jacard_coef_loss, metrics=[jacard_coef])

    filepath="/Users/anshugusain/tf-nuclei/Models/new_spectral-bestVAL/best_weights.h5"

    checkpoint = ModelCheckpoint(filepath, monitor='val_jacard_coef', verbose=1,save_best_only=True, mode='max')

    callbacks_list = [checkpoint]

    results = model.fit(X_train, y_train, batch_size = 1, steps_per_epoch=1, epochs=200, shuffle=False, validation_data=(X_test, y_test), callbacks=callbacks_list)

    model = tf.keras.models.load_model('/Users/anshugusain/tf-nuclei/Models/new_spectral-bestVAL/best_weights.h5')

    test_predict = model.predict(np.expand_dims(X_test[0],0))
    test_predict = np.asarray(test_predict)
    y_true.append(y_test[0])
    y_pred.append(test_predict)

[1 2 3 4] [0]
[[[[0.         0.         0.         0.         0.        ]
   [0.30538717 0.31345895 0.4263078  0.439762   0.49816117]
   [0.31141183 0.30950665 0.46455643 0.476542   0.49497485]
   ...
   [0.2836148  0.27390817 0.5465601  0.5826407  0.6202097 ]
   [0.27555776 0.27353248 0.5114999  0.5198174  0.59654623]
   [0.30802172 0.30749583 0.5240792  0.5430306  0.5859298 ]]

  [[0.33632326 0.32789823 0.49086568 0.49578816 0.49616325]
   [0.2804284  0.28632548 0.55365866 0.542335   0.4614363 ]
   [0.3063582  0.30564484 0.5155014  0.52404606 0.44545636]
   ...
   [0.27972388 0.28212067 0.48467803 0.48516005 0.54463077]
   [0.3016374  0.28881937 0.5449541  0.52954465 0.5343758 ]
   [0.3314613  0.34226534 0.6220699  0.63715595 0.54027843]]

  [[0.3064085  0.3083594  0.51673704 0.5349637  0.4931031 ]
   [0.26674464 0.26089224 0.6022772  0.59826624 0.47070685]
   [0.33659562 0.33073294 0.51886183 0.51825523 0.46763924]
   ...
   [0.37238362 0.37516975 0.5820721  0.5552577  0.5522353 ]
 

2023-07-31 20:17:36.538523: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-31 20:17:36.538548: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/200


/Users/anshugusain/miniforge3/envs/tf-nuclei/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2023-07-31 20:17:36.992555: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-07-31 20:17:37.614606: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: -0.1162 - jacard_coef: 0.1162
Epoch 1: val_jacard_coef improved from -inf to 0.13865, saving model to /Users/anshugusain/tf-nuclei/Models/new_spectral-bestVAL/best_weights.h5


2023-07-31 20:17:38.656852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: -0.1162 - jacard_coef: 0.1162 - val_loss: -0.1387 - val_jacard_coef: 0.1387
Epoch 2/200
1/1 [==============================] - ETA: 0s - loss: -0.1302 - jacard_coef: 0.1302
Epoch 2: val_jacard_coef improved from 0.13865 to 0.14749, saving model to /Users/anshugusain/tf-nuclei/Models/new_spectral-bestVAL/best_weights.h5
1/1 [==============================] - 0s 267ms/step - loss: -0.1302 - jacard_coef: 0.1302 - val_loss: -0.1475 - val_jacard_coef: 0.1475
Epoch 3/200
1/1 [==============================] - ETA: 0s - loss: -0.1318 - jacard_coef: 0.1318
Epoch 3: val_jacard_coef improved from 0.14749 to 0.14777, saving model to /Users/anshugusain/tf-nuclei/Models/new_spectral-bestVAL/best_weights.h5
1/1 [==============================] - 0s 270ms/step - loss: -0.1318 - jacard_coef: 0.1318 - val_loss: -0.1478 - val_jacard_coef: 0.1478
Epoch 4/200
1/1 [==============================] - ETA: 0s - loss: -0.1575 - jacard_coef: 0.1575
Epoch

In [16]:
test_predict = model.predict(np.expand_dims(X_test[0],0))

ValueError: in user code:

    File "/Users/anshugusain/miniforge3/envs/tf-nuclei/lib/python3.9/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/Users/anshugusain/miniforge3/envs/tf-nuclei/lib/python3.9/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/anshugusain/miniforge3/envs/tf-nuclei/lib/python3.9/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/anshugusain/miniforge3/envs/tf-nuclei/lib/python3.9/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/Users/anshugusain/miniforge3/envs/tf-nuclei/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/a

In [21]:
average = 0

for i in range(0,len(y_true)):
    average = jaccard_score((np.array(y_pred).squeeze())[i]>.3, np.array(y_true)[i], average='micro') + average
    print(average)


average / len(y_true)

0.6971584355667401
1.4457613749330775
2.1669896729179765
2.892386574218665
3.615717428622896


0.7231434857245792

In [22]:
X=[]
y=[]

for a in range(1,6):
  Xpath = '/Users/anshugusain/tf-nuclei/content/Stained-40X/Field' + str(a) + '/image.png'
  Ximg = tf.keras.preprocessing.image.load_img(Xpath)

  ypath = '/Users/anshugusain/tf-nuclei/content/Stained-40X/Field' + str(a) + '/Field' + str(a) + '-mask.tif'
  yimg = tf.keras.preprocessing.image.load_img(ypath, grayscale=True)

  Ximg = np.asarray(Ximg)
  Xpatches = patchify(Ximg,(HEIGHT,WIDTH,3), step=HEIGHT)
  yimg = np.asarray(yimg)  
  ypatches = patchify(yimg, (HEIGHT,WIDTH), step=WIDTH)

  for m in range (0,Xpatches.shape[0]):
    for l in range(0,Xpatches.shape[1]):
       y.append(ypatches[m,l])
       X.append(Xpatches[m, l,0])

X = np.array(X)
y = np.array(y)>0

X = (X/X.max())
y = (y/y.max())

/Users/anshugusain/miniforge3/envs/tf-nuclei/lib/python3.9/site-packages/keras/utils/image_utils.py:409: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn(


In [ ]:
y_true, y_pred = [], []

for i,j in cv.split(X):
    print(i,j)
    X_train, X_test = X[i,:],X[j,:]
    print(X_train)
    print(X_test)
    y_train, y_test = y[i], y[j]

    #Building the model

    inputs = tf.keras.layers.Input((HEIGHT, WIDTH, 3))
    s = inputs #converts integers from input layers into floating points

    #Contraction Path
                            # feature space  
                            # \/
    c1 = tf.keras.layers.Conv2D(16,(3,3), activation = "relu", kernel_initializer='he_normal', padding = 'same')(s)#kernel initializer is the initial weights used by the neural network (he_normal is a trunkated normal distribution centered on 0)
    c1 = tf.keras.layers.Dropout(.1)(c1)
    c1 = tf.keras.layers.Conv2D(16,(3,3), activation = "relu", kernel_initializer="he_normal", padding = "same")(c1 ) #Padding ensures that the output image has the same dimensions as the input image
    p1 = tf.keras.layers.MaxPooling2D((2,2))(c1) #First maxpooling layer puts the values into a 2x2 matrix, and gets the maximum values

    c2 = tf.keras.layers.Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p1)
    c2 = tf.keras.layers.Dropout(.1)(c2)
    c2 = tf.keras.layers.Conv2D(32,(3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c2)
    p2=tf.keras.layers.MaxPooling2D((2,2))(c2)

    c3= tf.keras.layers.Conv2D(64, (3,3), activation = "relu", kernel_initializer="he_normal", padding="same")(p2)
    c3 = tf.keras.layers.Dropout(.2)(c3)
    c3=tf.keras.layers.Conv2D(64,(3,3), activation = "relu", kernel_initializer="he_normal", padding="same")(c3)
    p3=tf.keras.layers.MaxPooling2D((2,2))(c3)

    c4= tf.keras.layers.Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p3)
    c4= tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3,3),activation="relu", kernel_initializer="he_normal", padding="same")(c4)
    p4=tf.keras.layers.MaxPooling2D((2,2))(c4)

    c5=tf.keras.layers.Conv2D(256, (3,3), activation ="relu", kernel_initializer="he_normal", padding="same")(p4)
    c5=tf.keras.layers.Dropout(0.3)(c5)
    c5=tf.keras.layers.Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c5)
    p5=tf.keras.layers.MaxPooling2D((2,2))(c5)

    c6=tf.keras.layers.Conv2D(512, (3,3), activation ="relu", kernel_initializer="he_normal", padding="same")(p5)
    c6=tf.keras.layers.Dropout(0.3)(c6)
    c6=tf.keras.layers.Conv2D(512, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c6)



    u8= tf.keras.layers.Conv2DTranspose(256, (2,2), strides=(2,2), padding="same")(c6)
    u8= tf.keras.layers.concatenate([u8,c5])#Concatenates layers u6 and c4 as shown in the Unet architecture
    c8=tf.keras.layers.Conv2D(256, (3,3), activation="relu", kernel_initializer='he_normal', padding="same")(u8)
    c8=tf.keras.layers.Dropout(0.2)(c8)
    c8=tf.keras.layers.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)


    u9= tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding="same")(c8)
    u9= tf.keras.layers.concatenate([u9,c4])#Concatenates layers u6 and c4 as shown in the Unet architecture
    c9=tf.keras.layers.Conv2D(128, (3,3), activation="relu", kernel_initializer='he_normal', padding="same")(u9)
    c9=tf.keras.layers.Dropout(0.2)(c9)
    c9=tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    u10= tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding="same")(c9)
    u10= tf.keras.layers.concatenate([u10,c3])#Concatenates layers u6 and c4 as shown in the Unet architecture
    c10=tf.keras.layers.Conv2D(64, (3,3), activation="relu", kernel_initializer='he_normal', padding="same")(u10)
    c10=tf.keras.layers.Dropout(0.2)(c10)
    c10=tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c10)

    u11=tf.keras.layers.Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(c10)
    u11=tf.keras.layers.concatenate([u11,c2])
    c11=tf.keras.layers.Conv2D(32,(3,3), activation="relu",kernel_initializer="he_normal", padding="same")(u11)
    c11=tf.keras.layers.Dropout(0.2)(c11)
    c11=tf.keras.layers.Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c11)

    u12=tf.keras.layers.Conv2DTranspose(16, (2,2),strides=(2,2), padding="same")(c11)
    u12=tf.keras.layers.concatenate([u12, c1])
    c12=tf.keras.layers.Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u12)
    c12=tf.keras.layers.Dropout(0.1)(c12)
    c12=tf.keras.layers.Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c12)


    outputs=tf.keras.layers.Conv2D(1, (1,1), activation="sigmoid")(c12)
    model=tf.keras.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(lr=.001), loss='binary_crossentropy', metrics=['accuracy'])

    filepath="/Users/anshugusain/tf-nuclei/Models/new_spectral-bestVAL/best_weights.h5"

    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,save_best_only=True, mode='max')

    callbacks_list = [checkpoint]

    results = model.fit(X_train, y_train, batch_size = 1, steps_per_epoch=1, epochs=200, shuffle=False, validation_data=(X_test, y_test), callbacks=callbacks_list)

    model = tf.keras.models.load_model('/Users/anshugusain/tf-nuclei/Models/new_spectral-bestVAL/best_weights.h5')

    test_predict = model.predict(np.expand_dims(X_test[0],0))
    test_predict = np.asarray(test_predict)
    y_true.append(y_test[0])
    y_pred.append(test_predict)

In [25]:
average = 0

for i in range(0,len(y_true)):
    average = jaccard_score((np.array(y_pred).squeeze())[i]>.5, np.array(y_true)[i], average='micro') + average
    print(average)


average / len(y_true)

0.7312091319686314
1.5269020457736677
2.32883254770417
3.11090792356764
3.854227176812261


0.7708454353624522